# gopigo3 model & navigation

The final objective of this chapter are:
- Create a model of our gopigo3 robot 
- create a model of the virtual environment
- locate the robot in our created environment
- use gazebo package to simulate the robot kinematics and dynamics
- use rviz package to view the different topics and nodes

The final model represents the real gopigo3 robot we will use in the laboratory

Interesting complementary information could be found:

https://brjapon.medium.com/learning-robotics-with-ros-made-easy-12197c918dab

http://wiki.ros.org/Robots/gopigo3

https://robots.ros.org/gopigo3/

https://github.com/ros-gopigo3/gopigo3


<img src="./Images/1_gopigo3_UB.png">

## 1. rUBot gopigo3 model generation

First of all, we have to create the "gopigo3_description" package containing the gopigo3 model:

In [ ]:
cd ~/gopigo_pc_ws/src
catkin_create_pkg gopigo3_description rospy
cd ..
catkin_make

Then open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/gopigo_pc_ws/devel/setup.bash

export ROS_IP=192.168.18.121 (specify here your VM IP)

export ROS_MASTER_URI=http://localhost:11311

To create our robot model, we use URDF files (Unified Robot Description Format). URDF file is an XML format file for representing a robot model.(http://wiki.ros.org/urdf/Tutorials)

We have created 2 folders for model description:
- URDF: folder where different URDF models are located
- meshes: folder where 3D body models in stl format are located.

The main parts of URDF model are:
- links: diferent bodies/plastic elements
- joints: connection between 2 links 
- sensors & actuators plugins (2D camera, LIDAR and DC motors)

The link definition contains:
- visual properties: the origin, geometry and material
- collision properties: the origin and geomnetry
- inertial properties: the origin, mass and inertia matrix

The joint definition contains:
- joint Type (fixed, continuous)
- parent and child frames
- origin frame
- rotation axis

In the case or wright wheel:

In [ ]:
<!-- Right Wheel -->
  <link name="right_wheel">
    <visual>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <geometry>
          <cylinder length="0.05" radius="0.1" />
      </geometry>
      <material name="orange"/>
    </visual>
    <collision>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <geometry>
          <cylinder length="0.05" radius="0.1" />
      </geometry>
    </collision>
    <inertial>
      <origin xyz="0 0 0" rpy="1.570795 0 0" />
      <mass value="0.15"/>
      <inertia ixx="0.01" ixy="0.0" ixz="0.0" iyy="0.005" iyz="0.0" izz="0.005"/>
    </inertial>
  </link>
  
  <!-- Right Wheel joint -->
  <joint name="joint_right_wheel" type="continuous">
    <parent link="base_link"/>
    <child link="right_wheel"/>
    <origin xyz="0 -0.30 0.025" rpy="0 0 0" /> 
    <axis xyz="0 1 0" />
  </joint>

The gopigo3 model includes different sensors and actuators:

Sensors:
- a two-dimensional camera: correspondas to RBPi camera
- a 360º LIDAR sensor: corresponds to the EAI YDLIDAR X4 (https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html)

Actuator:
- Differential drive actuator: based on 2 DC motors with encoders to obtain the Odometry information

The full model contains also information about the sensor and actuator controllers using specific Gazebo plugins (http://gazebosim.org/tutorials?tut=ros_gzplugins#Tutorial:UsingGazebopluginswithROS). 

Gazebo plugins give your URDF models greater functionality and can tie in ROS messages and service calls for sensor output and motor input. 

These plugins can be referenced through a URDF file, and to insert them in the URDF file, you have to follow the sintax:
- for Differential drive actuator:

In [ ]:
<robot>
  ... robot description ...
  <gazebo>
    <plugin name="differential_drive_controller" filename="libdiffdrive_plugin.so">
      ... plugin parameters ...
    </plugin>
  </gazebo>
  ... robot description ...
</robot>

<img src="./Images/1_Drive_kine.png">

- for sensor plugin:

In [ ]:
<robot>
  ... robot description ...
  <link name="sensor_link">
    ... link description ...
  </link>

  <gazebo reference="sensor_link">
    <sensor type="camera" name="camera1">
      ... sensor parameters ...
      <plugin name="camera_controller" filename="libgazebo_ros_camera.so">
        ... plugin parameters ..
      </plugin>
    </sensor>
  </gazebo>

</robot>

The diferent model files in urdf folder are:
- gopigo_basic.gazebo: includes only the geometrical description of gopigo robot.
- gopigo3_basic.gazebo: includes sensors and actuator:
    - the 2D camera and LDS sensors
    - the differential drive actuator
- gopigo3_init.gazebo: includes sensors and actuator:
    - the 2D camera, LDS and LIDAR sensors
    - the differential drive actuator
- gopigo3.gazebo: includes corrections in the proposed model:
    - base_link geometry and inertia
    - 2D camera real position
    - LIDAR scaling and orientation

We use a specific "gopigo3_rviz.launch" launch file where we specify the robot model we want to open in rviz with a configuration specified in "gopigo3_model.rviz":

In [ ]:
<?xml version="1.0"?>
<launch>
<!-- set these parameters on Parameter Server -->
   <param name="robot_description" textfile="$(find gopigo3_description)/urdf/gopigo_basic.gazebo" />
  <!-- send fake joint values -->
  <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
    <param name="use_gui" value="False"/>
  </node>
  <!-- Combine joint values -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"/>
  <!-- Show in Rviz   -->
  <node name="rviz" pkg="rviz" type="rviz"  args="-d $(find gopigo3_description)/rviz/gopigo3_model.rviz"/>
</launch>

Launch the ROS visualization tool to check that the model is properly built. 
RViz only represents the robot visual features. You have available all the options to check every aspect of the appearance of the model

In [ ]:
roslaunch gopigo3_description gopigo_basic_rviz.launch

<img src="./Images/1_gopigo_basic_rviz.png">

In [ ]:
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/1_gopigo3_rviz.png">

### gopigo3 model improvements
Some modifications in the initial model "gopigo3_init.gazebo" have been made to:
- locate the LIDAR in the correct position
- locate the camera axis in front position to see properly

Special attention has to be done to the YDLIDAR sensor.

- YDLIDAR X4 is a 360-degree two-dimensional rangefinder (hereinafter referred to as X4) developed by YDLIDAR team. 
- Based on the principle of triangulation, it is equipped with related optics, electronics and algorithm design to achieve high-frequency and high-precision distance measurement. 
- The mechanical structure rotates 360 degrees to continuously output the angle information as well as the point cloud data of the scanning environment while ranging.
-10m Ranging distance (2cm absolute error)

<img src="./Images/LP01_ydlidar.png">

It is important to note that:
- the number of points of real YDLidar is 720 (one each half degree)
- the number of points of simulated Lidar is 360 (one each degree)

### Exercise 1:

Modify the gopigo3.gazebo model to take into account:
- Adapt the base-link geometry and inertia to the real gopigo3 robot

<img src="./Images/1_mides.png">

## rUBot gopigo3 spawn in world environment

In robotics research, always before working with a real robot, we simulate the robot behaviour in a virtual environment close to the real one. The dynamic simulation of a robot, is a better approach to examining the actual behavior of the robot rather than just using software. Rigid body mechanics, including mass and inertia, friction, damping, motor controllers, sensor detection properties, noise signals, and every aspect of the robot and the environment that can be retained in a model with reasonable accuracy is much less expensive when replicated in a simulator than if you tried to do this with physical hardware.

Gazebo is an open source 3D robotics simulator and includes an ODE physics engine and OpenGL rendering, and supports code integration for closed-loop control in robot drives. This is sensor simulation and actuator control.

A specific ROS Package called "gazebo_control" have been created for this purpose.

In [ ]:
roslaunch gazebo_control spawn.launch
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/LP02_rubotcoop_spawn.png">

<img src="./Images/1_gopigo3_gazebo.png">

You can see the nodes and topics generated using rqt_graph

<img src="./Images/LP05_rqt_gopigo.png">

In order to kill the previous Gazebo process, type:

killall gzserver && killall gzclient

or type ctrl+r and kill

## 2. Design the Project world

Here we have first to design the project world, for exemple a maze from where our rUBot gopigo3 has to navigate autonomously.

There is a very useful and simple tool to design a proper world: Building editor" in gazebo.

Open gazebo as superuser:

In [ ]:
sudo gazebo

You can build your world using "Building Editor" in Edit menu

<img src="./Images/LP03_BuildingWorld1_1.png">

You can save:
- the generated model in a model folder (without extension)

Close the Builder Editor, modify the model position and add other elements if needed. Save:
- the generated world (with extension .world) in the world folder.

Once you finish is better to close the terminal you have work as superuser


### Exercise 2:
Generate a proper world defined by the schematic proposed below

<img src="./Images/1_maze.png">

### Spawn the gopigo robot

Now, spawn the rUBotCoop robot in our generated world. You have to create a "rubot_navigation.launch" file:

In [ ]:
<launch>
  <arg name="world" default="world1.world"/> 
  <arg name="model" default="gopigo3.gazebo" />
  <arg name="x_pos" default="0.0"/>
  <arg name="y_pos" default="0.0"/>
  <arg name="z_pos" default="0.0"/>

  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" value="$(find gazebo_control)/worlds/$(arg world)"/>
    <arg name="paused" value="false"/>
    <arg name="use_sim_time" value="true"/>
    <arg name="gui" value="true"/>
    <arg name="headless" value="false"/>
    <arg name="debug" value="false"/>
  </include>

  <param name="robot_description" textfile="$(find gopigo3_description)/urdf/$(arg model)" />
  
  <node pkg="gazebo_ros" type="spawn_model" name="spawn_urdf"
    args="-urdf -model gopigo3 -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos) -param robot_description" />

</launch>

In [ ]:
roslaunch gazebo_control rubot_navigation.launch

<img src="./Images/LP04_BuildingWorld1_2.png">

To control the robot with the Keyboard you have to install the "teleop-tools" package:

In [ ]:
sudo apt-get install ros-melodic-teleop-tools

Then you will be able to control the robot with the Keyboard typing:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

## 3. rUBot gopigo navigation control in the new world environment

Once the world has been generated we will create a ROS Package "rubot_control" to perform the autonomous navigation 

In [ ]:
cd ~/gopigo_pc_ws/src
catkin_create_pkg rubot_control rospy std_msgs sensor_msgs geometry_msgs nav_msgs
cd ..
catkin_make

<img src="./Images/LP07_rubot_control.png">

We will create now different navigation python files in "src" folder:
- move1_gopigo.py: to define a rubot movement with linear and angular speed
- move2_gopigo_param.py: to perform the same operation using params
- move3_gopigo_distance.py: to specify a maximum distance

Specific launch files have been created to launch the nodes and python files created above:

In [ ]:
roslaunch rubot_control rubot_move1.launch

In [ ]:
roslaunch rubot_control rubot_move2.launch

In [ ]:
roslaunch rubot_control rubot_move3.launch

<img src="./Images/1_rubot_move3.png">

## rUBot gopigo autonomous navigation and obstacle avoidance

In order to navigate autonomously and avoid obstacles, we have created diferent python files in "src" folder:
- rubot_LIDAR_Test.py: to test the LIDAR distance readings and angles
- rubot_autonomous_navigation1.py: to perform a simple algorithm for navigation with obstacle avoidance

we will create also a "launch" folder including the corresponding launch files:
- rubot_lidar_test.launch
- rubot_nav1.launch

#### 1. LIDAR test
We have created a world to test the rubot model. This world is based on a wall to verify that the LIDAR see the obstacle in the correct angle. We have to launch the "rubot_lidar_test.launch" file in the "rubot_control" package.

In [ ]:
roslaunch rubot_control rubot_lidar_test.launch

<img src="./Images/1_rubot_test.png">

#### 2. Autonomous navigation with obstacle avoidance
We will use now the created world to test the autonomous navigation with obstacle avoidance performance. 

We have to launch the "rubot_nav1.launch" file in the "rubot_control" package.

In [ ]:
roslaunch rubot_control rubot_nav1.launch

Careful:
- we have included in launch file: gazebo spawn, rviz visualization and rubot_nav node execution 
- In rviz you have to change the fixed frame to "odom" frame

<img src="./Images/LP08_rubot_nav_key.png">

The algorithm description functionality is:
- "rubot_autonomous_navigation1.py": The Python script makes the robot go forward. 
    - LIDAR is allways searching the closest distance and the angle
    - when this distance is lower than a threshold, the robot goes backward with angular speed in the oposite direction of the minimum distance angle.

The node and topics graph:

<img src="./Images/01_rubot_nav_rqt.png">

### Exercise 3:

Create another rubot_autonomous_navigation.py file to improve the navigation process of our gopigo3 rubot for exemple to follow the wall on the right side.